<a href="https://colab.research.google.com/github/carlamaziero/Reconhecimento_padroes/blob/main/Aula4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn import datasets
from sklearn.decomposition import PCA

In [2]:
iris = datasets.load_iris()
X = iris.data[:, :2]
y = iris.target

In [3]:
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsRegressor
import numpy as np

In [4]:
modelo = KNeighborsRegressor()
scores = cross_validate(modelo, X, y, scoring=make_scorer(mean_squared_error, squared=False))
print(scores['test_score'])
print(scores)
sempad = np.mean(scores['test_score'])
print(f"Sem padronização: {sempad}")

[0.06324555 0.45607017 0.67330033 0.51510517 0.69761498]
{'fit_time': array([0.01761913, 0.00076056, 0.00087571, 0.00998449, 0.00591612]), 'score_time': array([0.00159764, 0.01231766, 0.00127363, 0.02033401, 0.00153542]), 'test_score': array([0.06324555, 0.45607017, 0.67330033, 0.51510517, 0.69761498])}
Sem padronização: 0.48106724085424457


In [5]:
from sklearn.model_selection import GridSearchCV

parametros = {'n_neighbors': [1,3,5]}

modelo = GridSearchCV(KNeighborsRegressor(), parametros, scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False))
modelo.fit(X, y)

GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [1, 3, 5]},
             scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False))

In [6]:
modelo.cv_results_

{'mean_fit_time': array([0.00076632, 0.00044265, 0.0004683 ]),
 'std_fit_time': array([1.07401619e-04, 2.42955902e-05, 7.07863758e-05]),
 'mean_score_time': array([0.00113006, 0.00070744, 0.0007618 ]),
 'std_score_time': array([2.37061088e-04, 2.46259502e-05, 6.46731480e-05]),
 'param_n_neighbors': masked_array(data=[1, 3, 5],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 1}, {'n_neighbors': 3}, {'n_neighbors': 5}],
 'split0_test_score': array([-0.        , -0.        , -0.06324555]),
 'split1_test_score': array([-0.57735027, -0.50184844, -0.45607017]),
 'split2_test_score': array([-0.85634884, -0.75767676, -0.67330033]),
 'split3_test_score': array([-0.65828059, -0.55444333, -0.51510517]),
 'split4_test_score': array([-0.81649658, -0.704483  , -0.69761498]),
 'mean_test_score': array([-0.58169526, -0.50369031, -0.48106724]),
 'std_test_score': array([0.30816224, 0.26874152, 0.22812491]),
 'rank_test_score': ar

In [7]:
modelo.best_estimator_

KNeighborsRegressor()

In [8]:
from sklearn.metrics import mean_squared_error
from math import sqrt

ypred = modelo.predict(X)
mse = mean_squared_error(y, ypred)
sqrt(mse)

0.3278210894171799

In [9]:
scores = cross_validate(modelo, X, y, scoring=make_scorer(mean_squared_error, squared=False), return_estimator=True)
print(scores['test_score'])
sempad = np.mean(scores['test_score'])
print(f"Sem padronização: {sempad}")

[0.06324555 0.45607017 0.67330033 0.51510517 0.69761498]
Sem padronização: 0.48106724085424457


In [10]:
scores

{'fit_time': array([0.03079009, 0.02506828, 0.0245676 , 0.02608633, 0.0241518 ]),
 'score_time': array([0.00077009, 0.00077105, 0.00075531, 0.00112462, 0.00074935]),
 'estimator': [GridSearchCV(estimator=KNeighborsRegressor(),
               param_grid={'n_neighbors': [1, 3, 5]},
               scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False)),
  GridSearchCV(estimator=KNeighborsRegressor(),
               param_grid={'n_neighbors': [1, 3, 5]},
               scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False)),
  GridSearchCV(estimator=KNeighborsRegressor(),
               param_grid={'n_neighbors': [1, 3, 5]},
               scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False)),
  GridSearchCV(estimator=KNeighborsRegressor(),
               param_grid={'n_neighbors': [1, 3, 5]},
               scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False)),
  GridSearchCV(estimator=

In [11]:
for estimator in scores['estimator']:
    print(estimator.best_estimator_)

KNeighborsRegressor()
KNeighborsRegressor()
KNeighborsRegressor()
KNeighborsRegressor()
KNeighborsRegressor()


In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

parametros = {'n_neighbors': [7 ,9 ,11 ,13]}

GridSearchKNN = GridSearchCV(KNeighborsRegressor(), parametros, scoring='neg_root_mean_squared_error')

modelo = Pipeline([
    ("padronização", StandardScaler()),
    ("gsknn", GridSearchKNN)
])
scores = cross_validate(modelo, X, y, scoring=make_scorer(mean_squared_error, squared=False))
print(scores['test_score'])
compad = np.mean(scores['test_score'])
print(f"Com padronização: {compad}")

[0.0451754  0.44262667 0.62324584 0.50776287 0.69891156]
Com padronização: 0.46354446526594284


In [13]:
pipeline = Pipeline([
    ("padronização", StandardScaler()),
    ("knn", KNeighborsRegressor())
])

parametros = {'knn__n_neighbors': [7,9,11, 13]}

modelo = GridSearchCV(pipeline, parametros, scoring='neg_root_mean_squared_error')

scores = cross_validate(modelo, X, y, scoring=make_scorer(mean_squared_error, squared=False))
print(scores['test_score'])
compad = np.mean(scores['test_score'])
print(f"Com padronização: {compad}")

[0.0451754  0.44262667 0.63440455 0.50776287 0.68263196]
Com padronização: 0.4625202887734445
